# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [177]:
problem = '''You are in charge of the supply purchasing of a company that produces two kinds of drugs. 
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market. 

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales. 

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded. 
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available. 
Your company can also only store a limited amount of raw materials. 
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces. 
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively. 
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. '''

## 2. Generate the mathematical model

In [178]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [179]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Drug Production

**Parameters:**

* **Raw Materials:**
    * $R$: Set of raw materials, indexed by $r$
    * $P_r$: Price per unit of raw material $r$
    * $C_r$: Maximum storage capacity for raw material $r$
    * $E_r$:  Amount of active ingredient extracted per unit of raw material $r$ (nominal value)
    * $\Delta E_r$: Maximum deviation of active ingredient extraction for raw material $r$ (e.g., 0.005 for 0.5%, 0.02 for 2%) 

* **Drugs:**
    * $D$: Set of drugs, indexed by $d$
    * $S_d$: Selling price per unit of drug $d$
    * $A_d$: Required amount of active ingredient per unit of drug $d$
    * $L_d$: Labor hours required to produce one unit of drug $d$
    * $M_d$: Machine hours required to produce one unit of drug $d$

* **Production:**
    * $B$: Total budget for raw material purchase and drug production
    * $LH$: Total available labor hours
    * $MH$: Total available machine hours


**Decision Variables:**

* $X_r$: Amount of raw material $r$ to purchase 
* $Y_d$: Amount of drug $d$ to produce

**Objective Function (Maximize Profit):**

 $$Maximize\  \sum_{d \in D} S_d Y_d - \sum_{r \in R} P_r X_r - \sum_{d \in D} (L_d Y_d + M_d Y_d)$$

**Constraints:**

1. **Budget Constraint:** 
    $$\sum_{r \in R} P_r X_r + \sum_{d \in D} (L_d Y_d + M_d Y_d) \le B$$

2. **Storage Capacity Constraint:**
    $$X_r \le C_r  \quad \forall r \in R$$

3. **Labor Hour Constraint:**
    $$\sum_{d \in D} L_d Y_d \le LH$$

4. **Machine Hour Constraint:**
    $$\sum_{d \in D} M_d Y_d \le MH$$

5. **Active Ingredient Requirement Constraint (considering variation):**
    $$\sum_{r \in R} (E_r - \Delta E_r) X_r  \ge \sum_{d \in D} A_d Y_d $$

6. **Non-negativity Constraints:**
    $$X_r \ge 0 \quad \forall r \in R$$
    $$Y_d \ge 0 \quad \forall d \in D$$


**Explanation:**

* The objective function calculates the total profit by subtracting the raw material cost and operational costs from the revenue generated by selling the drugs.
* Constraint 1 ensures that the total cost of purchasing and production does not exceed the budget. 
* Constraint 2 ensures that the amount of each raw material purchased does not exceed its storage capacity. 
* Constraints 3 and 4 ensure that the labor and machine hours used for production do not exceed the available capacity.
* Constraint 5 is crucial, as it guarantees that enough active ingredient is available for drug production, even considering the worst-case scenario of ingredient extraction variation from each raw material.
* Constraints 6 ensures that the amount of raw materials purchased and the amount of drug produced are non-negative.

This optimization model allows you to determine the optimal amount of each raw material to purchase and the amount of each drug to produce to maximize your profit while adhering to budgetary, capacity, and, most importantly, active ingredient requirements. 


## 3. Generate the pyomo code

In [180]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [181]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data
raw_materials = {'RM1', 'RM2', 'RM3'}
drugs = {'DrugA', 'DrugB'}

price_rm = {'RM1': 5, 'RM2': 8, 'RM3': 6}  
capacity_rm = {'RM1': 100, 'RM2': 150, 'RM3': 120} 
extraction_rate = {'RM1': 0.1, 'RM2': 0.15, 'RM3': 0.12}
deviation_extraction = {'RM1': 0.005, 'RM2': 0.01, 'RM3': 0.008}

selling_price = {'DrugA': 20, 'DrugB': 25}
active_ingredient_req = {'DrugA': 0.05, 'DrugB': 0.08}
labor_hours = {'DrugA': 2, 'DrugB': 2.5}
machine_hours = {'DrugA': 1.5, 'DrugB': 1.8}

budget = 2000
total_labor_hours = 500
total_machine_hours = 400

# Model
model = pyo.ConcreteModel()

# Sets
model.R = pyo.Set(initialize=raw_materials)
model.D = pyo.Set(initialize=drugs)

# Parameters
model.P = pyo.Param(model.R, initialize=price_rm)
model.C = pyo.Param(model.R, initialize=capacity_rm)
model.E = pyo.Param(model.R, initialize=extraction_rate)
model.Delta_E = pyo.Param(model.R, initialize=deviation_extraction)

model.S = pyo.Param(model.D, initialize=selling_price)
model.A = pyo.Param(model.D, initialize=active_ingredient_req)
model.L = pyo.Param(model.D, initialize=labor_hours)
model.M = pyo.Param(model.D, initialize=machine_hours)

model.B = pyo.Param(initialize=budget)
model.LH = pyo.Param(initialize=total_labor_hours)
model.MH = pyo.Param(initialize=total_machine_hours)

# Decision Variables
model.X = pyo.Var(model.R, within=pyo.NonNegativeReals)
model.Y = pyo.Var(model.D, within=pyo.NonNegativeReals)

# Objective Function
def profit(model):
    return sum(model.S[d] * model.Y[d] for d in model.D) - sum(model.P[r] * model.X[r] for r in model.R) - sum((model.L[d] + model.M[d]) * model.Y[d] for d in model.D)
model.profit = pyo.Objective(rule=profit, sense=pyo.maximize)

# Constraints
def budget_constraint(model):
    return sum(model.P[r] * model.X[r] for r in model.R) + sum((model.L[d] + model.M[d]) * model.Y[d] for d in model.D) <= model.B
model.budget_constraint = pyo.Constraint(rule=budget_constraint)

def storage_constraint(model, r):
    return model.X[r] <= model.C[r]
model.storage_constraint = pyo.Constraint(model.R, rule=storage_constraint)

def labor_constraint(model):
    return sum(model.L[d] * model.Y[d] for d in model.D) <= model.LH
model.labor_constraint = pyo.Constraint(rule=labor_constraint)

def machine_constraint(model):
    return sum(model.M[d] * model.Y[d] for d in model.D) <= model.MH
model.machine_constraint = pyo.Constraint(rule=machine_constraint)

def active_ingredient_constraint(model):
    return sum((model.E[r] - model.Delta_E[r]) * model.X[r] for r in model.R) >= sum(model.A[d] * model.Y[d] for d in model.D)
model.active_ingredient_constraint = pyo.Constraint(rule=active_ingredient_constraint)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Display results
print(results)
print("Profit:", pyo.value(model.profit))

print("Raw Material Purchase:")
for r in model.R:
    print(f"{r}: {pyo.value(model.X[r])}")

print("\nDrug Production:")
for d in model.D:
    print(f"{d}: {pyo.value(model.Y[d])}")
``` 


## 4. Input problem data and try running the generated code

In [184]:
import pyomo.environ as pyo

# Sample Data - Replace with actual data
raw_materials = {'RM1', 'RM2'}
drugs = {'DrugA', 'DrugB'}

price_rm = {'RM1': 100, 'RM2': 199.90}  
capacity_rm = {'RM1': 500, 'RM2': 500} 
extraction_rate = {'RM1': 0.01, 'RM2': 0.02}
deviation_extraction = {'RM1': 0.005, 'RM2': 0.02}

selling_price = {'DrugA': 6200, 'DrugB': 6900}
active_ingredient_req = {'DrugA': 0.5, 'DrugB': 0.6}
labor_hours = {'DrugA': 90, 'DrugB': 100}
machine_hours = {'DrugA': 40, 'DrugB': 50}

budget = 100000
total_labor_hours = 2000
total_machine_hours = 800

# Model
model = pyo.ConcreteModel()

# Sets
model.R = pyo.Set(initialize=raw_materials)
model.D = pyo.Set(initialize=drugs)

# Parameters
model.P = pyo.Param(model.R, initialize=price_rm)
model.C = pyo.Param(model.R, initialize=capacity_rm)
model.E = pyo.Param(model.R, initialize=extraction_rate)
model.Delta_E = pyo.Param(model.R, initialize=deviation_extraction)

model.S = pyo.Param(model.D, initialize=selling_price)
model.A = pyo.Param(model.D, initialize=active_ingredient_req)
model.L = pyo.Param(model.D, initialize=labor_hours)
model.M = pyo.Param(model.D, initialize=machine_hours)

model.B = pyo.Param(initialize=budget)
model.LH = pyo.Param(initialize=total_labor_hours)
model.MH = pyo.Param(initialize=total_machine_hours)

# Decision Variables
model.X = pyo.Var(model.R, within=pyo.NonNegativeReals)
model.Y = pyo.Var(model.D, within=pyo.NonNegativeReals)

# Objective Function
def profit(model):
    return sum(model.S[d] * model.Y[d] for d in model.D) - sum(model.P[r] * model.X[r] for r in model.R) - sum((model.L[d] + model.M[d]) * model.Y[d] for d in model.D)
model.profit = pyo.Objective(rule=profit, sense=pyo.maximize)

# Constraints
def budget_constraint(model):
    return sum(model.P[r] * model.X[r] for r in model.R) + sum((model.L[d] + model.M[d]) * model.Y[d] for d in model.D) <= model.B
model.budget_constraint = pyo.Constraint(rule=budget_constraint)

def storage_constraint(model, r):
    return model.X[r] <= model.C[r]
model.storage_constraint = pyo.Constraint(model.R, rule=storage_constraint)

def labor_constraint(model):
    return sum(model.L[d] * model.Y[d] for d in model.D) <= model.LH
model.labor_constraint = pyo.Constraint(rule=labor_constraint)

def machine_constraint(model):
    return sum(model.M[d] * model.Y[d] for d in model.D) <= model.MH
model.machine_constraint = pyo.Constraint(rule=machine_constraint)

def active_ingredient_constraint(model):
    return sum((model.E[r] - model.Delta_E[r]) * model.X[r] for r in model.R) >= sum(model.A[d] * model.Y[d] for d in model.D)
model.active_ingredient_constraint = pyo.Constraint(rule=active_ingredient_constraint)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Display results
print(results)
print("Profit:", pyo.value(model.profit))

print("Raw Material Purchase:")
for r in model.R:
    print(f"{r}: {pyo.value(model.X[r])}")

print("\nDrug Production:")
for d in model.D:
    print(f"{d}: {pyo.value(model.Y[d])}")

(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo

Problem: 
- Name: unknown
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 4
  Number of nonzeros: 13
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04039430618286133
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit: 0.0
Raw Material Purchase:
RM2: 0.0
RM1: 0.0

Drug Production:
DrugA: 0.0
DrugB: 0.0


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.

In [185]:
print(response.text)

## Mathematical Optimization Model for Drug Production

**Parameters:**

* **Raw Materials:**
    * $R$: Set of raw materials, indexed by $r$
    * $P_r$: Price per unit of raw material $r$
    * $C_r$: Maximum storage capacity for raw material $r$
    * $E_r$:  Amount of active ingredient extracted per unit of raw material $r$ (nominal value)
    * $\Delta E_r$: Maximum deviation of active ingredient extraction for raw material $r$ (e.g., 0.005 for 0.5%, 0.02 for 2%) 

* **Drugs:**
    * $D$: Set of drugs, indexed by $d$
    * $S_d$: Selling price per unit of drug $d$
    * $A_d$: Required amount of active ingredient per unit of drug $d$
    * $L_d$: Labor hours required to produce one unit of drug $d$
    * $M_d$: Machine hours required to produce one unit of drug $d$

* **Production:**
    * $B$: Total budget for raw material purchase and drug production
    * $LH$: Total available labor hours
    * $MH$: Total available machine hours


**Decision Variables:**

* $X_r$: Amount of ra

In [186]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data
raw_materials = {'RM1', 'RM2', 'RM3'}
drugs = {'DrugA', 'DrugB'}

price_rm = {'RM1': 5, 'RM2': 8, 'RM3': 6}  
capacity_rm = {'RM1': 100, 'RM2': 150, 'RM3': 120} 
extraction_rate = {'RM1': 0.1, 'RM2': 0.15, 'RM3': 0.12}
deviation_extraction = {'RM1': 0.005, 'RM2': 0.01, 'RM3': 0.008}

selling_price = {'DrugA': 20, 'DrugB': 25}
active_ingredient_req = {'DrugA': 0.05, 'DrugB': 0.08}
labor_hours = {'DrugA': 2, 'DrugB': 2.5}
machine_hours = {'DrugA': 1.5, 'DrugB': 1.8}

budget = 2000
total_labor_hours = 500
total_machine_hours = 400

# Model
model = pyo.ConcreteModel()

# Sets
model.R = pyo.Set(initialize=raw_materials)
model.D = pyo.Set(initialize=drugs)

# Parameters
model.P = pyo.Param(model.R, initialize=price_rm)
model.C = pyo.Param(model.R, initialize=capacity_rm)
model.E = pyo.Param(model.R, initialize=extraction_rate)
model.Delta_E = pyo.Param(model.R, initialize=deviation_extraction)

model.S = pyo.Para